# Intractive Deforestation and Biodiversity Ananlysis Dasboard

In [1]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.express as px
import pandas as pd

# Load dataset
df = pd.read_csv("merged_deforestation_biodiversity_india_full.csv")

# Initialize Dash app
app = dash.Dash(__name__, external_stylesheets=[
    'https://cdn.jsdelivr.net/npm/bootstrap@5.3.0/dist/css/bootstrap.min.css'
])

# App layout
app.layout = html.Div([
    html.Div([
        html.H1("Deforestation & Biodiversity Dashboard", className="text-center text-white mb-4 fw-bold"),
    ], className="bg-primary py-4 shadow-sm"),

    html.Div([
        html.Div([
            html.H3("Total Forest Area (%)", className="m-0 fs-5"),
            html.P(id="total-forest", className="fs-4 fw-bold m-0"),
        ], className="card p-3 shadow-sm bg-success text-white"),

        html.Div([
            html.H3("Biodiversity Index", className="m-0 fs-5"),
            html.P(id="biodiversity-index", className="fs-4 fw-bold m-0"),
        ], className="card p-3 shadow-sm bg-danger text-white"),
    ], className="d-flex justify-content-around mb-4 gap-4"),

    html.Div([
        html.Div([
            html.Label("Select State:", className="fw-bold mb-2"),
            dcc.Dropdown(
                id='state-dropdown',
                options=[{'label': state, 'value': state} for state in df['State'].unique()],
                value=df['State'].unique()[0],
                className='form-control'
            ),
        ], className="col-md-4"),

        html.Div([
            html.Label("Filter by Year:", className="fw-bold mb-2"),
            dcc.RangeSlider(
                id='year-slider',
                min=df['Year'].min(),
                max=df['Year'].max(),
                marks={str(year): str(year) for year in range(df['Year'].min(), df['Year'].max()+1, 5)},
                value=[df['Year'].min(), df['Year'].max()],
                className="mt-2"
            ),
        ], className="col-md-4"),
    ], className="row mb-4 p-3 bg-white rounded shadow-sm"),

    html.Div([
        dcc.Graph(id='forest-trend', className='bg-white p-3 rounded shadow-sm'),
    ], className="mb-4"),

    html.Div([
        html.Div([
            dcc.Graph(id='deforestation-bar', className='bg-white p-3 rounded shadow-sm'),
        ], className="col-md-6"),

        html.Div([
            dcc.Graph(id='biodiversity-pie', className='bg-white p-3 rounded shadow-sm'),
        ], className="col-md-6"),
    ], className="row mb-4"),

    html.Div([
        dcc.Graph(id='deforestation-heatmap', className='bg-white p-3 rounded shadow-sm'),
    ], className="mb-4"),
], className="container-fluid p-4 bg-light")

# Callbacks
@app.callback(
    [Output('forest-trend', 'figure'),
     Output('deforestation-bar', 'figure'),
     Output('biodiversity-pie', 'figure'),
     Output('deforestation-heatmap', 'figure'),
     Output('total-forest', 'children'),
     Output('biodiversity-index', 'children')],
    [Input('state-dropdown', 'value'),
     Input('year-slider', 'value')]
)
def update_graphs(selected_state, selected_years):
    state_data = df[(df['State'] == selected_state) &
                    (df['Year'] >= selected_years[0]) &
                    (df['Year'] <= selected_years[1])]

    # Forest Area Trend
    fig1 = px.line(state_data, x='Year', y='Forest_Area_Percentage', markers=True,
                    title=f"Forest Area (%) Trend in {selected_state}", labels={'Forest_Area_Percentage': 'Forest Area (%)'})


    # Transform data for deforestation causes
    cause_columns = ['Agriculture', 'Urbanization', 'Mining', 'Logging']
    melted_data = state_data.melt(id_vars=['Year'], value_vars=cause_columns, var_name='Cause', value_name='Forest_Loss_Amount')

    # Deforestation Causes Bar Chart
    fig2 = px.bar(
        melted_data, x='Cause', y='Forest_Loss_Amount', color='Cause',
        title=f"Deforestation Causes in {selected_state}",
        labels={'Forest_Loss_Amount': 'Deforested Area (sq km)'}
    )
    # Biodiversity Pie Chart
    fig3 = px.pie(state_data, names='Biodiversity_Impact', values='Total_Forest_Loss',title=f"Biodiversity Loss in {selected_state}")
    

    # Deforestation Heatmap
    # Filter dataset for heatmap
    heatmap_data = df[(df['Year'] >= selected_years[0]) & (df['Year'] <= selected_years[1])]
    
    # Ensure the correct column names are used
    fig4 = px.density_heatmap(
        heatmap_data, 
        x='Year', 
        y='Forest_Type', 
        z='Total_Forest_Loss',   
        color_continuous_scale='Reds',
        title=f"Deforestation Heatmap Across {selected_state}",
        labels={'Total_Forest_Loss': 'Deforested Area (sq km)'}
    )

    
    total_forest = f"{state_data['Forest_Area_Percentage'].mean():.2f}%"
    biodiversity_index = f"{state_data['Biodiversity_Index'].mean():.2f}"
    
    return fig1, fig2, fig3, fig4, total_forest, biodiversity_index

# Run app
if __name__ == '__main__':
    app.run(debug=True)
